# API Tests

In [15]:
import random, string

def gen_url():
    letters = string.ascii_lowercase
    protocols = ["http://", "https://", "ftp://"]
    level1s = [".com", ".ru", ".fr", ".uk"]
    domain = ''.join(random.choice(letters) for i in range(8))
    path = ''.join(random.choice(letters) for i in range(15))
    return random.choice(protocols) + domain + random.choice(level1s) + "/" + path

## 1. GRPC

### 1.1 Set up

In [16]:
!python -m grpc_tools.protoc -I../api/proto/ --python_out=. --pyi_out=. --grpc_python_out=. ../api/proto/links.proto

In [17]:
import grpc
import links_pb2_grpc

channel = grpc.insecure_channel('localhost:50051')
stub = links_pb2_grpc.LinkServiceStub(channel)

In [31]:
import links_pb2
import re

status_pattern = re.compile("StatusCode.([A-Z_]+)\n")

def extract_error(string):
    return status_pattern.search(string).group(1)

def create_link_grpc(url):
    try:
        request = links_pb2.CreateLinkRequest(url=str(url))
        result = stub.CreateLink(request)
        return result.alias
    except grpc.RpcError as e:
        return extract_error(str(e))
        

def get_link_grpc(alias):
    try:
        request = links_pb2.GetLinkRequest(alias=str(alias))
        result = stub.GetLink(request)
        return result.url
    except grpc.RpcError as e:
        return extract_error(str(e))

### 1.2 Run tests

In [43]:
!cd .. && sudo docker compose -f golinkcut_postgres-compose.yml run -d --service-ports golinkcut /go/bin/golinkcut --db-host golinkcut_db --validater

Error response from daemon: driver failed programming external connectivity on endpoint golinkcut-golinkcut-run-5a25ccae8baa (2697fbe1aeb992df9315a9be78bd0111520979dfae6ed35a18c7b81982278435): Bind for 0.0.0.0:50051 failed: port is already allocated
Error: failed to start containers: 5f0e8606579a211ae69065d392036d8f253fc89efce58adc2cedda551d9b6c87


In [42]:
print("trying normal tests:")
ok = True
for i in range(15):
    url = gen_url()
    alias = create_link_grpc(url)
    url_got = get_link_grpc(alias)
    if url != url_got:
        ok = False
        print(f"Got wrong results:\nurl:{url}\nalias:{alias}\ngot url:{url_got}")
        break
if ok:
    print("OK")

print("tests with errors:")

ok = True
url = gen_url()
alias = create_link_grpc(url)
result = create_link_grpc(url)
if create_link_grpc(url) == "ALREADY_EXISTS":
    print("OK")
else:
    print(f"Got wrong results:\nurl:{url}\nalias:{alias}\nresult:{result}")

result = get_link_grpc("0123456789")
if result == "NOT_FOUND":
    print("OK")
else:
    print("Wrong! Should be NOT_FOUND")

result = create_link_grpc("not-a-url-at-all")
print(result)

trying normal tests:
OK
tests with errors:
OK
OK
Ufrcg1ykbn


In [45]:
!cd .. && sudo docker compose -f golinkcut_postgres-compose.yml down

[+] Running 1/0
 ! Network golinkcut_default  Resource is s...                             0.0s 
